# Figure 12: Sensitivity to `bad_multiplier` (Discretion sweep)

Uses `artifacts/critique_discretion/discretion_sweep_summary.csv` and run-level `sim_paths.npz` files.

In [ ]:
import os, sys
import pathlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def _find_project_root():
    here = pathlib.Path.cwd().resolve()
    for p in [here, *here.parents]:
        if (p / "src").is_dir():
            return p
    cand = pathlib.Path("/content/econml")
    if (cand / "src").is_dir():
        return cand
    raise RuntimeError("Could not find project root containing src/.")

PROJECT_ROOT = _find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

SWEEP_ROOT = pathlib.Path(os.environ.get("CRITIQUE_SWEEP_ROOT", str(PROJECT_ROOT / "artifacts" / "critique_discretion")))
summary_path = SWEEP_ROOT / "discretion_sweep_summary.csv"
print("SWEEP_ROOT:", SWEEP_ROOT)
print("summary:", summary_path)
if not summary_path.exists():
    raise FileNotFoundError("Sweep summary missing. Run scripts/train_discretion_uncertainty_sweep.py first.")

df = pd.read_csv(summary_path)
if "variant" in df.columns:
    df["variant"] = df["variant"].astype(str).str.upper()
if "bad_multiplier" not in df.columns:
    # fallback parser from scenario slug, e.g. variant_A_bm_1p25
    def _parse_bm(s):
        t = str(s).split("_bm_")[-1]
        return float(t.replace("p", "."))
    df["bad_multiplier"] = df["scenario"].map(_parse_bm)

df = df.sort_values(["variant", "bad_multiplier"]).reset_index(drop=True)
df.head()

In [ ]:
def _sim_metrics(run_dir: str):
    p = pathlib.Path(run_dir) / "sim_paths.npz"
    if not p.exists():
        return {"pi_std": np.nan, "i_std": np.nan, "pi_std_bad": np.nan, "i_std_bad": np.nan}
    sim = np.load(p)
    pi = np.asarray(sim["pi"]).reshape(-1)
    i = np.asarray(sim["i"]).reshape(-1) if "i" in sim.files else np.full_like(pi, np.nan)
    s = np.asarray(sim["s"]).reshape(-1)
    bad = (s == 1)
    out = {
        "pi_std": float(np.nanstd(pi)),
        "i_std": float(np.nanstd(i)),
        "pi_std_bad": float(np.nanstd(pi[bad])) if np.any(bad) else np.nan,
        "i_std_bad": float(np.nanstd(i[bad])) if np.any(bad) else np.nan,
    }
    return out

mets = [ _sim_metrics(rd) for rd in df["run_dir"].tolist() ]
mdf = pd.DataFrame(mets)
df = pd.concat([df, mdf], axis=1)
df.head()

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(13, 9), constrained_layout=True)
plot_spec = [
    ("best_loss", "Best training loss"),
    ("train_rms", "Train residual RMS"),
    ("pi_std_bad", "Std(inflation), bad regime"),
    ("i_std_bad", "Std(nominal rate), bad regime"),
]
for ax, (col, title) in zip(axes.ravel(), plot_spec):
    for var, g in df.groupby("variant"):
        g = g.sort_values("bad_multiplier")
        ax.plot(g["bad_multiplier"], g[col], marker="o", label=f"Variant {var}")
    ax.set_title(title)
    ax.set_xlabel("bad_multiplier")
    ax.grid(alpha=0.25)

axes[0, 0].legend(loc="best")
fig.suptitle("Discretion sensitivity to bad_multiplier (A vs B)")
out_dir = PROJECT_ROOT / "artifacts" / "figures"
out_dir.mkdir(parents=True, exist_ok=True)
out_path = out_dir / "fig12_discretion_bad_multiplier_sensitivity.png"
fig.savefig(out_path, dpi=160)
print("Saved:", out_path)
plt.show()